# Analyse du traitement des données

In [1]:
import re
import rdflib
import time
import csv
import json
import Levenshtein
import random
import itertools

Le but des deux fonctions suivantes est de nettoyer les chaines de caractères, de supprimer les caractères non-ascii
ou de les remplacer par leurs equivalents

In [2]:
def nettoyer_unicode(c):
    liste_codes = {
        'Ã\xa0': 'à',
        'Ã€': 'À',
        'Ã¢': 'â',
        'Ã‚': 'Â',
        'Ã©': 'é',
        'Ã\x89': 'É',
        'Ã\xa8': 'è',
        'Ã\xaa': 'ê',
        'Ã\x8a': 'Ê',
        'Ã«': 'ë',
        'Ã®': 'î',
        'Ã\x8e': 'Î',
        'Ã¯': 'ï',
        'Ã´': 'ô',
        'Ã\x94': 'Ô',
        'Ã¹': 'ù',
        'Ã»': 'û',
        'Å\x93': 'œ',
        'Â«': '«',
        'Â»': '»',
        'Ã§': 'ç',
        'Ã\x87': 'Ç',
        'Âº': 'º',
        'â\x80\x99': '’',
        'â\x80\xa6': '…',
    }

    for code in liste_codes:
        c = c.replace(code, liste_codes[code])

    return c


def nettoyer_accents(c):
    """
    remplace les caractères et accents par leurs équivalents sans accents
    :param c: str
    :return: même str sans accents
    """
    liste_codes = {
        'ã': 'a',
        'ã'.upper(): 'A',
        'â': 'a',
        'Â': 'A',
        'à': 'a',
        'À': 'A',
        'ä': 'a',
        'Ä': 'A',
        'é': 'e',
        'É': 'E',
        'è': 'e',
        'È': 'E',
        'ê': 'e',
        'Ê': 'E',
        'ï': 'i',
        'Ï': 'I',
        'î': 'i',
        'Î': 'I',
        'ô': 'o',
        'Ô': 'O',
        'ö': 'o',
        'Ö': 'O',
        'ù': 'u',
        'Ù': 'U',
        'ü': 'u',
        'Ü': 'U',
        'û': 'u',
        'Û': 'U',
        'ÿ': 'y',
        'Ÿ': 'y',
        'ç': 'c',
        'œ': 'oe',
        '\'': ' ',
        '"': '',
    }

    try:
        for code in liste_codes:
            c = c.replace(code, liste_codes[code])

        return c

    except:
        return None

Fonction qui retire toutes les informations à l'interieur des parenthèses.
Nous devons utiliser une boucle car les parenthèses peuvent être imbriquées
ex: titre_de_l_oeuvre ( nom_auteur (principale))

In [3]:
def remove_text_between_parentheses(text):
    n = 1  # run at least once
    while n:
        text, n = re.subn(r'\([^()]*\)', '', text)  # remove non-nested/flat balanced parts
    return text

- On retire les caractères non-ascii et accents
- On retire les parenthèses et leurs contenus
- On supprimes tout les caractères non-alpha-numériques, et les doubles espaces, sauf le caractère "-", 
nécessaire pour les noms composés
- On passe la chaine de caractère en lettres minuscules

In [4]:
def normalize(string):
    """
    normalise une chaine de caractère pour faciliter leurs comparaisons
    :param string: chaine de caractère à normaliser
    :return: même chaine de caractère normalisée
    """

    if isinstance(string, str) and string:
        string = nettoyer_accents(nettoyer_unicode(string))
        string = remove_text_between_parentheses(string)
        string = ' '.join(re.sub(r'[^\w\s\-]', ' ', string).split()).lower()
        
        return string
    else:
        return None

On normalise la chaine de caractère et on supprimer les lettres seules
ex: A. Nom_de_famille => Nom_de_famille

In [5]:
def normalize_author(string):
    """
    Processus de normalisation des auteurs,
    permet de retirer les lettres isolées
    :param string: chaine de caractère a traiter
    :return: chaine de caractère traitée
    """
    if isinstance(string, str) and string:
        return re.sub(r'^\w\s|\s\w\s|\s\w$', '', normalize(string))
    else:
        return None

- On retire le texte entre parenthèse
- on supprime les carctères autre que numérique et 'X'
- Si l'isbn est un isbn 10, on le transforme en isbn 13 en ajoutant '978' au début
- On supprime le dernier chiffre et on le remplace par X (chiffre de controle). Si un chiffre manque, on l'ajoute simplement

In [6]:
def normalize_isbn(isbn):
    """
    Retire tout les autres symboles que les chiffres
    :param isbn: chaine de caractère
    :return: chaine de caratère
    """
    isbn = remove_text_between_parentheses(isbn)
    isbn = re.sub(r'[^\dX]', '', isbn)
    if len(isbn) <= 10:
        isbn = '978' + isbn
    if len(isbn) == 13 or len(isbn) == 10:
        isbn = isbn[:-1] + 'X'
    elif len(isbn) == 12 or len(isbn) == 9:
        isbn = isbn + 'X'
    return isbn

## Analyse du traitement des données ADP

In [7]:
start_loading_data_time = time.time()
# Loading des données sauvegardées dans la mémoire ram
g_book_ADP = rdflib.Graph()
g_author_ADP = rdflib.Graph()
ADP_book_graph = g_book_ADP.parse("../Graphes/grapheADPLivres.rdf")
ADP_author_graph = g_author_ADP.parse("../Graphes/grapheADPAuteurs.rdf")

ADP_books = []
# Analyse des données ADP à partir des graphs
# parcour des triplets du graph
for subj, pred in g_book_ADP.subject_predicates(rdflib.URIRef("http://www.sogides.com/classe/Livre")):
    book_ADP = g_book_ADP.predicate_objects(subj)
    book_ADP_resume = { 'id': subj.n3(), "data_base": "ADP",
                        'title': '', 'author': [], 'isbn': [],
                        'title_raw': '', 'author_raw': [], 'isbn_raw': []}

    for info in book_ADP:
        if info[0] == rdflib.URIRef("https://schema.org/name"):
            if not info[1].n3():
                print("problème !!! infoADP: ", info)
            book_ADP_resume['title'] = normalize(info[1])
            book_ADP_resume['title_raw'] = info[1]
        elif info[0] == rdflib.URIRef("https://schema.org/author"):
            author_ADP = g_author_ADP.predicate_objects(info[1])
            for author_info in author_ADP:
                if author_info[0] == rdflib.URIRef("https://schema.org/name"):
                    book_ADP_resume['author_raw'].append(author_info[1].n3())
                    if author_info[1].n3().replace("\"", ""):
                        book_ADP_resume['author'].append(normalize_author(author_info[1].n3()))
        elif info[0] == rdflib.URIRef("https://schema.org/isbn"):
            if isinstance(info[1].n3(), list):
                print("stop")
            book_ADP_resume['isbn_raw'].append(info[1].n3())
            book_ADP_resume['isbn'].append(normalize_isbn(info[1].n3()))
    ADP_books.append(book_ADP_resume)

ADP_loading_time = time.time()
print("ADP_loading_time: ", ADP_loading_time - start_loading_data_time)

ADP_loading_time:  17.803131580352783


### titre
Pour le titre on normalise simplement:

In [8]:
for exemple in ADP_books:
    if "(" in exemple["title_raw"]:
        print("titre intial:", exemple["title_raw"], " titre traité: ", exemple["title"] )
        break

titre intial: TOUR DE GUET (LA) T03  titre traité:  tour de guet t03


### auteur
Pour les auteurs, on normalise en utilisant la fonction de normalisation. Il n'y a pas besoin de traiter plus loin,
car la séparation des auteurs est déjà effectuée dans le graph. On retire aussi les caractères '"' qui sont redondants

In [9]:
for exemple in ADP_books:
    good_exemple = False
    if len(exemple["author"]) > 1:
        for author in exemple["author"]:
            if '-' in author:
                good_exemple = True
    if good_exemple:            
        print("auteur(s) initial(s): ", exemple["author_raw"], "auteur(s) traité: ", exemple["author"])
        break

auteur(s) initial(s):  ['"Laurens Van Den Muyzenberg"', '"Sa Sainteté Le Dalaï-Lama"'] auteur(s) traité:  ['laurens van den muyzenberg', 'sa saintete le dalai-lama']


### isbn
Pour ADP, on sait que l'isbn est unique et qu'il sont tous des isbn 13.
En revanche certains manquent le chiffre de contrôle. On se contente de le normaliser

In [10]:
for exemple in ADP_books:
    if len(exemple["isbn_raw"][0].replace("\"", "")) == 12:
        print("exemple d'isbn avec le chiffre de controle manquant: \nisbn initial(s): ", exemple["isbn_raw"], "isbn traité: ", exemple["isbn"])
        break
for exemple in ADP_books:
    if len(exemple["isbn_raw"][0].replace("\"", "")) == 13 and exemple["isbn_raw"][0].replace("\"", "")[-1] != 'X':
        print("exemple d'isbn 13 normalisé: \nisbn initial(s): ", exemple["isbn_raw"], "isbn traité: ", exemple["isbn"])
        break

exemple d'isbn avec le chiffre de controle manquant: 
isbn initial(s):  ['"629048046827"'] isbn traité:  ['629048046827X']
exemple d'isbn 13 normalisé: 
isbn initial(s):  ['"9782760401310"'] isbn traité:  ['978276040131X']


## Analyse du traitement des données Depot_legal

In [11]:
start_loading_data_time = time.time()
g_item_DL = rdflib.Graph()
book_graph_DL = g_item_DL.parse("../Graphes/grapheDepotLegal.rdf")

DL_books = []

for subj, pred in g_item_DL.subject_predicates(rdflib.URIRef("http://dbpedia.org/ontology/Book")):
    book_DL = g_item_DL.predicate_objects(subj)
    book_DL_resume = { 'id': subj.n3(), "data_base": "Depot_legal",
                        'title': '', 'author': [], 'isbn': [],
                        'title_raw': '', 'author_raw': [], 'isbn_raw': []}
    for info in book_DL:
        if info[0] == rdflib.URIRef("https://schema.org/name"):
            if info[1].n3():
                book_DL_resume['title'] = normalize(info[1])
                book_DL_resume['title_raw'] = info[1]
            else:
                print("gros probleme DL: ", subj)
        elif info[0] == rdflib.URIRef("https://schema.org/author"):
            familyName = None
            givenName = None
            familyName_raw = None
            givenName_raw = None
            author_ADP = g_item_DL.predicate_objects(info[1])
            for author_info in author_ADP:
                if author_info[0] == rdflib.URIRef("https://schema.org/givenName"):
                    givenName = normalize_author(author_info[1].n3().replace("\"", "")) if author_info[1].n3() else None
                    givenName_raw = author_info[1].n3()
                elif author_info[0] == rdflib.URIRef("https://schema.org/familyName"):
                    familyName = normalize_author(author_info[1].n3().replace("\"", "")) if author_info[1].n3() else None
                    familyName_raw = author_info[1].n3()
            if familyName and givenName:
                book_DL_resume['author'].append(" ".join([givenName, familyName]))
                book_DL_resume['author_raw'].append(" ".join([givenName_raw, familyName_raw]))
            elif familyName:
                book_DL_resume['author'].append(familyName)
                book_DL_resume['author_raw'].append(familyName)
            elif givenName:
                book_DL_resume['author'].append(givenName)
                book_DL_resume['author_raw'].append(givenName)

        elif info[0] == rdflib.URIRef("https://schema.org/isbn"):
            book_DL_resume['isbn'].append(normalize_isbn(info[1].n3()))
            book_DL_resume['isbn_raw'].append(info[1].n3())

    DL_books.append(book_DL_resume)
    
DL_loading_time = time.time()
print("DL_loading_time: ", DL_loading_time - start_loading_data_time)

DL_loading_time:  73.84276819229126


### titre
Pour le titre on normalise simplement
On peut remaquer quelques problèmes, par exemples nous n'avons pas le titre 
pour certains livres

In [12]:
for exemple in DL_books:
    if "(" in exemple["title_raw"] and exemple["title"]:
        print("exemple simple: titre intial:", exemple["title_raw"], " titre traité: ", exemple["title"] )
        break
for exemple in DL_books:
    if not exemple["title"]:
        print("titre non existant: titre intial:", exemple["title_raw"], " titre traité: ", exemple["title"] )
        break

exemple simple: titre intial: La chambre noire (incubo)  titre traité:  la chambre noire
titre non existant: titre intial: (K)  titre traité:  


### auteur
Pour les auteurs, on normalise en utilisant la fonction de normalisation. Il n'y a pas besoin de traiter plus loin,
car la séparation des auteurs est déjà effectuée dans le graph. On retire aussi les caractères '"' qui sont redondants
En revanche, Dépot légal fait la différence entre le nom de famille et le prénom, ce que ne font pas les autres bases
de donnée. On va donc simplement concaténer les chaines de caractères avec un espace entre les noms et prénoms

In [13]:
for exemple in DL_books:
    good_exemple = False
    if len(exemple["author"]) > 1:
        for author in exemple["author"]:
            if '-' in author:
                good_exemple = True
    if good_exemple:            
        print("auteur(s) initial(s): ", exemple["author_raw"], "auteur(s) traité: ", exemple["author"])
        break

auteur(s) initial(s):  ['"Anik" "Lessard"', '"Marie-Claude" "Martel"'] auteur(s) traité:  ['anik lessard', 'marie-claude martel']


### isbn
Pour Dépot legal, on sait que l'isbn peut être multiple et de forme variée,
 mais la séparation s'est faite dans la création du graph. On se contente de normaliser
 On peut remarque que certains livres ont une liste conséquente d'isbn

In [14]:
for exemple in DL_books:
    if len(exemple["isbn_raw"][0].replace("\"", "")) == 13 and exemple["isbn_raw"][0].replace("\"", "")[-1] != 'X':
        print("exemple d'isbn 13 normalisé: \nisbn initial(s): ", exemple["isbn_raw"], "isbn traité: ", exemple["isbn"])
        break
for exemple in DL_books:
    if len(exemple["isbn_raw"][0].replace("\"", "")) == 10 and exemple["isbn_raw"][0].replace("\"", "")[-1] != 'X':
        print("exemple d'isbn 10 normalisé: \nisbn initial(s): ", exemple["isbn_raw"], "isbn traité: ", exemple["isbn"])
        break
for exemple in DL_books:
    if len(exemple["isbn_raw"]) > 60:
        print("exemple de livre avec plus de 60 isbn \nisbn initial(s): ", str(exemple["isbn_raw"])[0:100] + "...", "isbn traité: ", str(exemple["isbn"])[0:10] + "...")
        print('nb d\'isbn: ', len(exemple["isbn_raw"]))
        break

exemple d'isbn 13 normalisé: 
isbn initial(s):  ['"9782896514168"'] isbn traité:  ['978289651416X']
exemple d'isbn 10 normalisé: 
isbn initial(s):  ['"2896380639"'] isbn traité:  ['978289638063X']
exemple de livre avec plus de 60 isbn 
isbn initial(s):  ['"2895433917"', '"2895432287"', '"289543543X"', '"9782895435440"', '"2895433097"', '"2895432295"', ... isbn traité:  ['97828954...
nb d'isbn:  65


## Analyse du traitement des données ILE

In [15]:
start_loading_data_time = time.time()
g_item_ILE = rdflib.Graph()
item_graph_ILE = g_item_ILE.parse("../Graphes/grapheILE.rdf")
book_graph_DL = g_item_DL.parse("../Graphes/grapheDepotLegal.rdf")

ILE_books = []

for subj, pred in g_item_ILE.subject_predicates(rdflib.URIRef("http://recif.litterature.org/ontologie/classe/oeuvre")):
    # count_book += 1
    ILE_book = g_item_ILE.predicate_objects(subj)
    book_ILE_resume = { 'id': subj.n3(), "data_base": "ILE",
                        'title': '', 'author': [], 'isbn': [],
                        'title_raw': '', 'author_raw': [], 'isbn_raw': []}
    for info in ILE_book:
        if info[0] == rdflib.URIRef("https://schema.org/name"):
            if info[1].n3():
                book_ILE_resume['title'] = normalize(info[1])
                book_ILE_resume['title_raw'] = info[1]
            else:
                print("gros probleme ILE: ", subj)
        elif info[0] == rdflib.URIRef("https://schema.org/author"):
            author_ILE = g_item_ILE.predicate_objects(info[1])
            for author_info in author_ILE:
                if author_info[0] == rdflib.URIRef("https://schema.org/name"):
                    book_ILE_resume['author_raw'].append(author_info[1].n3())
                    if author_info[1].n3().replace("\"", ""):
                        book_ILE_resume['author'].append(normalize_author(author_info[1].n3()))
        elif info[0] == rdflib.URIRef("https://schema.org/isbn"):
            book_ILE_resume['isbn_raw'].append(info[1].n3())
            if normalize_isbn(info[1].n3()).replace("\"", "")\
                and len(normalize_isbn(info[1].n3()).replace("\"", "")) >= 8:
                for isbn in re.split("[|;,]", info[1].n3().replace("\"", "")):
                    book_ILE_resume['isbn'].append(normalize_isbn(isbn))

    ILE_books.append(book_ILE_resume)
    
ILE_loading_time = time.time()
print("ILE_loading time: ", ILE_loading_time - start_loading_data_time)

ILE_loading time:  80.57568097114563


### titre
Pour le titre on normalise simplement
On peut remaquer quelques problèmes, par exemples nous n'avons pas le titre 
pour certains livres

In [16]:
for exemple in ILE_books:
    if "(" in exemple["title_raw"] and exemple["title"]:
        print("exemple simple: titre intial:", exemple["title_raw"], " titre traité: ", exemple["title"] )
        break
for exemple in ILE_books:
    if not exemple["title"]:
        print("titre non existant: titre intial:", exemple["title_raw"], " titre traité: ", exemple["title"] )
        break

exemple simple: titre intial: Les confitures de coings et autres textes (1972, 1971)  titre traité:  les confitures de coings et autres textes
titre non existant: titre intial: (Parenthèses)  titre traité:  


### auteur
Pour les auteurs, on normalise en utilisant la fonction de normalisation. Il n'y a pas besoin de traiter plus loin,
car la séparation des auteurs est déjà effectuée dans le graph. On retire aussi les caractères '"' qui sont redondants

In [17]:
for exemple in ILE_books:
    good_exemple = False
    for author in exemple["author"]:
        if '-' in author:
            good_exemple = True
    if good_exemple:            
        print("auteur(s) initial(s): ", exemple["author_raw"], "auteur(s) traité: ", exemple["author"])
        break

auteur(s) initial(s):  ['"Bernier, Jovette-Alice"'] auteur(s) traité:  ['bernier jovette-alice']


### isbn
Pour ILE, on sait que l'isbn peut être multiple et de forme variée 
(séparé par des caractères: "-", et suivit de parenthèses), une première division s'est faite dans le graph
mais cela demande une deuxième passe, notament pour les séparateurs "|" et ";". On normalise par la suite.
On peut remarque que certains livres ont une liste conséquente d'isbn

In [18]:
for exemple in ILE_books:
    if len(exemple["isbn_raw"][0].replace("\"", "").replace("-", "")) == 13 and exemple["isbn_raw"][0].replace("\"", "")[-1] != 'X':
        print("exemple d'isbn 13 normalisé: \nisbn initial(s): ", exemple["isbn_raw"], "isbn traité: ", exemple["isbn"])
        break
for exemple in ILE_books:
    if len(exemple["isbn_raw"][0].replace("\"", "").replace("-", "")) == 10 and exemple["isbn_raw"][0].replace("\"", "")[-1] != 'X':
        print("exemple d'isbn 10 normalisé: \nisbn initial(s): ", exemple["isbn_raw"], "isbn traité: ", exemple["isbn"])
        break
for exemple in ILE_books:
    good_example = False
    for isbn in exemple["isbn_raw"]:
        if "|" in isbn:
            good_example = True
    if good_example:
        print("exemple d'isbn séparable par '|': ", str(exemple["isbn_raw"])[0:100] + "...", "isbn traité: ", str(exemple["isbn"])[0:10] + "...")
        print('nb d\'isbn: ', len(exemple["isbn_raw"]))
        break
for exemple in ILE_books:
    if len(exemple["isbn"]) > 10:
        print("exemple de livre avec plus de 10 isbn \nisbn initial(s): ", str(exemple["isbn_raw"])[0:100] + "...", "isbn traité: ", str(exemple["isbn"])[0:10] + "...")
        print('nb d\'isbn: ', len(exemple["isbn"]))
        break

exemple d'isbn 13 normalisé: 
isbn initial(s):  ['"9782764436387"'] isbn traité:  ['978276443638X']
exemple d'isbn 10 normalisé: 
isbn initial(s):  ['"2-922109-89-5"'] isbn traité:  ['978292210989X']
exemple d'isbn séparable par '|':  ['"978-2-922327-41-0 (br.)|2-922327-41-8"']... isbn traité:  ['97829223...
nb d'isbn:  1
exemple de livre avec plus de 10 isbn 
isbn initial(s):  ['"9782896951949 (cycle 1, livre 1) ; 9782896951956 (cycle 1, livre 2) ; 9782896952489 (cycle 1, liv... isbn traité:  ['97828969...
nb d'isbn:  14


## Analyse du traitement des données d'Hurtubise

In [19]:
books_Hurtubise_file = open("./Hurtubise/Exportation-Hurtubise.csv", "r", encoding='ISO-8859-1')
csv_reader = csv.DictReader(books_Hurtubise_file, delimiter=',', fieldnames=[
    "Editeur", "ISBN Papier", "ISBN PDF", "ISBN epub", "Titre", "Sous - titre", "Titre de la serie",
    "Contributeurs", "Contributeur(premier)", "Langue", "Langue Origine", "Resume", "Nombre de pages",
    "Date de parution", "Annee de parution", "Sujet  THEMA principal", "Sujet THEMA",
    "Quantificateur Georaphique", "Quantificateur de langue", "Quantificateur Historique", "Niveau soclaire FR",
    "Niveau scolaire QC", "Cycle scolaire FR", "Niveau de lecture", "Echele CECR", "Quantificateur d'interet",
    "Quantificateur d'age", "Quantificateur de style", "Classification Editoriale", "Mots cles"

])
Hurtubise_books = []
first_line = next(csv_reader)
for book in csv_reader:
    book_Hurtubise_resume = { 'id':None, "data_base": "Hurtubise",
                        'title': '', 'author': [], 'isbn': [],
                        'title_raw': '', 'author_raw': [], 'isbn_raw': []}

    for key, value in book.items():
        if value:
            if key == 'Titre':
                book_Hurtubise_resume['title'] = normalize(value)
                book_Hurtubise_resume['title_raw'] = value

            elif key == 'Contributeurs':
                book_Hurtubise_resume['author_raw'].append(value.split(','))
                for author in value.split(','):
                    book_Hurtubise_resume['author'].append(normalize_author(author))

            elif key == 'ISBN Papier' or key == 'ISBN PDF' or key == 'ISBN epub':
                if key == 'ISBN Papier':
                    book_Hurtubise_resume['id'] = value

                book_Hurtubise_resume['isbn'].append(normalize_isbn(value))
                book_Hurtubise_resume['isbn_raw'].append(value)

    Hurtubise_books.append(book_Hurtubise_resume)
books_Hurtubise_file.close()

### titre
Pour le titre on normalise simplement
On peut remaquer quelques problèmes, par exemple nous n'avons pas le titre 
pour certains livres

In [20]:
for exemple in Hurtubise_books:
    if "(" in exemple["title_raw"] and exemple["title"]:
        print("exemple simple: titre intial:", exemple["title_raw"], " titre traité: ", exemple["title"] )
        break
for exemple in Hurtubise_books:
    if not exemple["title"]:
        print("titre non existant: titre intial:", exemple["title_raw"], " titre traité: ", exemple["title"] )
        break

exemple simple: titre intial: Selfies et sushis (coffret)  titre traité:  selfies et sushis
titre non existant: titre intial:   titre traité:  


### auteur
Pour les auteurs, on normalise en utilisant la fonction de normalisation. Il n'y a pas besoin de traiter plus loin,
car la séparation des auteurs est déjà effectuée dans le graph. On peut remarquer que beaucoup d'information
sur le status des auteurs est contenue dans les parenthèses

In [21]:
for exemple in Hurtubise_books:
    good_exemple = False
    if len(exemple["author"]) > 1:
        for author in exemple["author"]:
            if '-' in author:
                good_exemple = True
    if good_exemple:            
        print("auteur(s) initial(s): ", exemple["author_raw"], "auteur(s) traité: ", exemple["author"])
        break

auteur(s) initial(s):  [['David Skuy (De (auteur))', ' Michel Rudel-Tessier (Traduit par)']] auteur(s) traité:  ['david skuy', 'michel rudel-tessier']


### isbn
Pour Hurtubise, on sait que l'isbn peut être multiple, mais toujours de taille 13. On normalise donc simplement.
On peut remarque que certains livres ont plusieurs isbns, mais pas plus de 3

In [22]:
for exemple in Hurtubise_books:
    if len(exemple["isbn_raw"][0].replace("\"", "")) == 13 and exemple["isbn_raw"][0].replace("\"", "")[-1] != 'X':
        print("exemple d'isbn 13 normalisé: \nisbn initial(s): ", exemple["isbn_raw"], "isbn traité: ", exemple["isbn"])
        break
for exemple in Hurtubise_books:
    if len(exemple["isbn_raw"]) > 2:
        print("exemple de livre avec plus de 60 isbn \nisbn initial(s): ", str(exemple["isbn_raw"])[0:100] + "...", "isbn traité: ", str(exemple["isbn"])[0:10] + "...")
        print('nb d\'isbn: ', len(exemple["isbn_raw"]))
        break

exemple d'isbn 13 normalisé: 
isbn initial(s):  ['9782897815189', '9782896473304', '9782896476503'] isbn traité:  ['978289781518X', '978289647330X', '978289647650X']
exemple de livre avec plus de 60 isbn 
isbn initial(s):  ['9782897815189', '9782896473304', '9782896476503']... isbn traité:  ['97828978...
nb d'isbn:  3


## Analyse du traitement des données de Babelio

In [23]:
babelioJsonBooks = open("./Babelio/babelio_livres.json", "r")
Babelio_books = []

for book in json.load(babelioJsonBooks):
    book_Babelio_resume = { 'id':None, "data_base": "Babelio",
                        'title': '', 'author': [], 'isbn': [],
                        'title_raw': '', 'author_raw': [], 'isbn_raw': []}

    for key, value in book.items():
        if value:
            if key == 'titre':
                book_Babelio_resume['title'] = normalize(value)
                book_Babelio_resume['title_raw'] = value

            elif key == 'auteur':
                book_Babelio_resume['author_raw'].append(value)
                for author in value:
                    book_Babelio_resume['author'].append(normalize_author(author))

            elif key == 'isbn':
                book_Babelio_resume['isbn'].append(normalize_isbn(value))
                book_Babelio_resume['isbn_raw'].append(value)

            elif key == 'url':
                book_Babelio_resume['id'] = value

    Babelio_books.append(book_Babelio_resume)
babelioJsonBooks.close()

### titre
Pour le titre on normalise simplement

In [24]:
for exemple in Babelio_books:
    if "(" in exemple["title_raw"] and exemple["title"]:
        print("exemple simple: titre intial:", exemple["title_raw"], " titre traité: ", exemple["title"] )
        break

exemple simple: titre intial:  Les fleurs sauvages du Québec (tome 2)   titre traité:  les fleurs sauvages du quebec


### auteur
Pour les auteurs, on normalise en utilisant la fonction de normalisation. Il n'y a pas besoin de traiter plus loin.
On peut remarque que certains livres comportent une quantité conséquente d'auteur

In [25]:
for exemple in Babelio_books:
    good_exemple = False
    if len(exemple["author"]) > 3:
        for author in exemple["author"]:
            if '-' in author:
                good_exemple = True
    if good_exemple:            
        print("auteur(s) initial(s): ", exemple["author_raw"], "auteur(s) traité: ", exemple["author"])
        break

auteur(s) initial(s):  [['Maxime  Robin', 'Jean-Michel  Girouard', 'Erika  Soucis', 'Jocelyn  Pelletier', "Noémie  O'Farrell", 'Sophie  Grenier-Héroux', 'Lorraine  Côté']] auteur(s) traité:  ['maxime robin', 'jean-michel girouard', 'erika soucis', 'jocelyn pelletier', 'noemiefarrell', 'sophie grenier-heroux', 'lorraine cote']


### isbn
Pour Babelio, on sait que l'isbn est unique et de la forme isbn 13 (car EAN)

In [26]:
for exemple in Babelio_books:
    if len(exemple["isbn_raw"][0].replace("\"", "")) == 13 and exemple["isbn_raw"][0].replace("\"", "")[-1] != 'X':
        print("exemple d'isbn 13 normalisé: \nisbn initial(s): ", exemple["isbn_raw"], "isbn traité: ", exemple["isbn"])
        break

exemple d'isbn 13 normalisé: 
isbn initial(s):  ['9782892618617'] isbn traité:  ['978289261861X']


# Comparaison des livres

In [ ]:
all_books = ADP_books + ILE_books + Hurtubise_books + Babelio_books + DL_books
random.shuffle(all_books)
couple_books = itertools.combinations(all_books, 2)

## isbn

In [28]:
def compare_book_by_isbn(book1, book2):
    isbn_boolean = True
    isbn_in_common = False
    if book1["isbn"] and book2["isbn"]:
        isbn_boolean = False
        isbn_in_common = False
        for isbn1 in book1["isbn"]:
            for isbn2 in book2["isbn"]:
                if compare_isbn(isbn1, isbn2):
                    isbn_boolean = True
                    isbn_in_common = True
    return isbn_boolean, isbn_in_common

def compare_isbn(isbn1, isbn2):
    """
    verifie si deux isbns sont identiques à partir de leur chaine de caractère
    cette fonction comporte la logique de la comparaison, les données sont normalement déjà formatées
    :param isbn1: 1er isbn
    :param isbn2: 2eme isbn
    :return: boolean
    """
    return isbn1 == isbn2


for book1, book2 in couple_books:
    isbn_boolean, isbn_in_common = compare_book_by_isbn(book1, book2)
    if isbn_in_common: 
        print("exemple de concordance d'isbn => isbn1: ", book1["isbn_raw"], " isbn2: ", book2["isbn_raw"])
        break
for book1, book2 in couple_books:
    isbn_boolean, isbn_in_common = compare_book_by_isbn(book1, book2)
    if isbn_boolean:
        print("exemple de manque d'information d'isbn => isbn1: ", book1["isbn_raw"], " isbn2: ", book2["isbn_raw"])
        break
for book1, book2 in couple_books:
    isbn_boolean, isbn_in_common = compare_book_by_isbn(book1, book2)
    if not isbn_boolean:
        print("exemple d'isbn différents => isbn1: ", book1["isbn_raw"], " isbn2: ", book2["isbn_raw"])
        break

exemple de concordance d'isbn => isbn1:  ['"2-89021-652-7 (br.)"']  isbn2:  ['"2890216527"']
exemple de manque d'information d'isbn => isbn1:  ['"2-89021-652-7 (br.)"']  isbn2:  ['""']
exemple d'isbn différents => isbn1:  ['"2-89021-652-7 (br.)"']  isbn2:  ['"9782897740870"', '"9782897740887"']


## auteur

In [29]:
def compare_book_by_author(book1, book2):
    author_boolean = True
    author_in_common = False
    if book1['author'] and book2['author']:
        author_boolean = False
        author_in_common = False
        for author1 in book1['author']:
            for author2 in book2['author']:
                if compare_authors(author1, author2):
                    author_in_common = True
                    author_boolean = True
    return author_boolean, author_in_common

def compare_authors(author1, author2):
    """
    verifie si deux auteurs sont identiques à partir de leur nom
    cette fonction comporte la logique de la comparaison, les données sont normalement déjà formatées
    :param author1: nom du premier auteur
    :param author2: nom du second auteur
    :return: boolean
    """
    if not author1 or not author2:
        return False
    author1 = author1.split(" ")
    author2 = author2.split(" ")
    return True if list(set(author1) & set(author2)) else False

for book1, book2 in couple_books:
    author_boolean, author_in_common = compare_book_by_isbn(book1, book2)
    if author_in_common: 
        print("exemple de concordance d'auteurs => isbn1: ", book1["isbn_raw"], " isbn2: ", book2["isbn_raw"])
        break
for book1, book2 in couple_books:
    author_boolean, author_in_common = compare_book_by_isbn(book1, book2)
    if author_boolean:
        print("exemple de manque d'information des auteurs => isbn1: ", book1["isbn_raw"], " isbn2: ", book2["isbn_raw"])
        break
for book1, book2 in couple_books:
    author_boolean, author_in_common = compare_book_by_isbn(book1, book2)
    if not author_boolean:
        print("exemple d'auteurs différents => isbn1: ", book1["isbn_raw"], " isbn2: ", book2["isbn_raw"])
        break

exemple de concordance d'auteurs => isbn1:  ['9782897237363', '9782897237370', '9782897237387']  isbn2:  ['"9782897237363"']
exemple de manque d'information des auteurs => isbn1:  ['9782897237363', '9782897237370', '9782897237387']  isbn2:  ['" Rel"']
exemple d'auteurs différents => isbn1:  ['9782897237363', '9782897237370', '9782897237387']  isbn2:  ['"9782896861484"']


## titres

In [30]:
def compare_book_by_title(book1, book2):
    dist_titre = Levenshtein.distance(book1['title'], book2['title'])
    dist_bool = dist_titre < max(1, min(len(book1['title']), len(book2['title'])) / 4)
    dist_bool_neg = dist_titre >= min(len(book1['title']), len(book2['title'])) / 2
    return dist_bool, dist_bool_neg


for book1, book2 in couple_books:
    dist_bool, dist_bool_neg = compare_book_by_isbn(book1, book2)
    if dist_bool: 
        print("exemple de concordance de titre => isbn1: ", book1["isbn_raw"], " isbn2: ", book2["isbn_raw"])
        break
for book1, book2 in couple_books:
    dist_bool, dist_bool_neg = compare_book_by_isbn(book1, book2)
    if not dist_bool_neg and dist_bool:
        print("exemple de manque d'information des titres => isbn1: ", book1["isbn_raw"], " isbn2: ", book2["isbn_raw"])
        break
for book1, book2 in couple_books:
    dist_bool, dist_bool_neg = compare_book_by_isbn(book1, book2)
    if dist_bool_neg:
        print("exemple de titres différents => isbn1: ", book1["isbn_raw"], " isbn2: ", book2["isbn_raw"])
        break

exemple de concordance de titre => isbn1:  ['9782897237363', '9782897237370', '9782897237387']  isbn2:  []
exemple de manque d'information des titres => isbn1:  ['9782897237363', '9782897237370', '9782897237387']  isbn2:  ['""']
exemple de titres différents => isbn1:  ['9782897237363', '9782897237370', '9782897237387']  isbn2:  ['"9782897237387"', '"9782897237370"']


# Analyse de la décision
## Comparaison des livres sur plusieurs critères

In [31]:
def compare_books(book1, book2):
    """
    verifie si deux livres sont identiques à partir de leur informations
    cette fonction comporte la logique de la comparaison, les données sont normalement déjà formatées
    :param book1: (title: str, author: [str,...], isbns: [str,...])
    :param book2: (title: str, author: [str,...], isbns: [str,...])
    :return: boolean
    """
    
    if not book1['title'] or not book2['title']:
        return {"conclusion": False,
            "isbn egaux et titre similaire": False,
            "isbn egaux mais titre legerement differents": False,
            "isbn egaux mais titre très differents": False,
            "isbn different mais titres equivalents": False,
            "isbn different mais titre et auteurs egaux": False,
            "titre equivalents mais auteurs differents": False,
            "titre egaux mais pas d'infos en plus": False,
            "titre et auteurs egaux": False}
    
    isbn_boolean, isbn_in_common = compare_book_by_isbn(book1, book2)

    author_boolean ,author_in_common = compare_book_by_author(book1, book2)

    dist_bool ,dist_bool_neg = compare_book_by_title(book1, book2)

    return {"conclusion": (dist_bool and author_in_common) or isbn_in_common,
            "isbn egaux et titre similaire": isbn_in_common and dist_bool,
            "isbn egaux mais titre legerement differents": isbn_in_common and not dist_bool and not dist_bool_neg,
            "isbn egaux mais titre très differents": isbn_in_common and dist_bool_neg,
            "isbn different mais titres equivalents": not isbn_boolean and dist_bool and author_boolean and not author_in_common,
            "isbn different mais titre et auteurs egaux": not isbn_boolean and dist_bool and author_in_common,
            "titre equivalents mais auteurs differents": isbn_boolean and not isbn_in_common and dist_bool and not author_boolean,
            "titre egaux mais pas d'infos en plus": isbn_boolean and not isbn_in_common and dist_bool and author_boolean and not author_in_common,
            "titre et auteurs egaux": isbn_boolean and not isbn_in_common and dist_bool and author_in_common}

for book1, book2 in couple_books:
    results = compare_books(book1, book2)
    if results["isbn egaux et titre similaire"] == True:
        print("isbn egaux et titre similaire titre1: ", book1['title_raw'], book1['title'], ' titre2: ', book2['title'], ' isbn: ',
            book1['isbn_raw'], ' isbn: ', book2['isbn_raw'], " auteur1 ", book1["author_raw"], " auteur2 ", book2["author_raw"])
        break
for book1, book2 in couple_books:
    results = compare_books(book1, book2)
    if results["isbn egaux mais titre legerement differents"] == True:
        print("isbn egaux mais titre legerement differents titre1: ", book1['title'], ' titre2: ', book2['title'], ' isbn: ',
            book1['isbn_raw'], ' isbn: ', book2['isbn_raw'], " auteur1 ", book1["author_raw"], " auteur2 ", book2["author_raw"])
        break
for book1, book2 in couple_books:
    results = compare_books(book1, book2)
    if results["isbn egaux mais titre très differents"] == True:
        print("isbn egaux mais titre très differents titre1: ", book1['title'], ' titre2: ', book2['title'], ' isbn: ',
            book1['isbn_raw'], ' isbn: ', book2['isbn_raw'], " auteur1 ", book1["author_raw"], " auteur2 ", book2["author_raw"])
        break
for book1, book2 in couple_books:
    results = compare_books(book1, book2)
    if results["isbn different mais titres equivalents"] == True:
        print("isbn different mais titres equivalents titre1: ", book1['title'], ' titre2: ', book2['title'], ' isbn: ',
            book1['isbn_raw'], ' isbn: ', book2['isbn_raw'], " auteur1 ", book1["author_raw"], " auteur2 ", book2["author_raw"])
        break
for book1, book2 in couple_books:
    results = compare_books(book1, book2)
    if results["isbn different mais titre et auteurs egaux"] == True:
        print("isbn different mais titre et auteurs egaux titre1: ", book1['title'], ' titre2: ', book2['title'], ' isbn: ',
            book1['isbn_raw'], ' isbn: ', book2['isbn_raw'], " auteur1 ", book1["author_raw"], " auteur2 ", book2["author_raw"])
        break
for book1, book2 in couple_books:
    results = compare_books(book1, book2)
    if results["titre equivalents mais auteurs differents"] == True:
        print("titre equivalents mais auteurs differents titre1: ", book1['title'], ' titre2: ', book2['title'], ' isbn: ',
            book1['isbn_raw'], ' isbn: ', book2['isbn_raw'], " auteur1 ", book1["author_raw"], " auteur2 ", book2["author_raw"])
        break
for book1, book2 in couple_books:
    results = compare_books(book1, book2)
    if results["titre egaux mais pas d'infos en plus"] == True:
        print("titre equivalent mais pas d'infos en plus titre1: ", book1['title'], ' titre2: ', book2['title'], ' isbn: ',
            book1['isbn_raw'], ' isbn: ', book2['isbn_raw'], " auteur1 ", book1["author_raw"], " auteur2 ", book2["author_raw"])
        break
for book1, book2 in couple_books:
    results = compare_books(book1, book2)
    if results["titre et auteurs egaux"] == True:
        print("titre equivalent et auteurs egaux titre1: ", book1['title'], ' titre2: ', book2['title'], ' isbn: ',
            book1['isbn_raw'], ' isbn: ', book2['isbn_raw'], " auteur1 ", book1["author_raw"], " auteur2 ", book2["author_raw"])
        break

isbn egaux et titre similaire titre1:  Zazou zazou  titre2:  zazou  isbn:  ['"978-2-89595-322-7"']  isbn:  ['"9782895953227"']  auteur1  ['"Perreault, Valérie"']  auteur2  ['"Valérie" "Perreault"']
isbn egaux mais titre legerement differents titre1:  la tour de guet  titre2:  tour de guet t01  isbn:  ['"9782896513932"', '"9782896952571"']  isbn:  ['"9782896513932"']  auteur1  ['"Ève" "Patenaude"']  auteur2  ['"Ã\x88VE PATENAUDE"']
isbn egaux mais titre très differents titre1:  l epidemie  titre2:  l epidemie histoires de deserteurs  isbn:  ['"2760400980"']  isbn:  ['"9782760400986"']  auteur1  ['"Major, André"']  auteur2  ['"André Major"']
isbn different mais titres equivalents titre1:  traverses  titre2:  traversees  isbn:  ['"978-2-89634-077-4"']  isbn:  ['"9782921956321"']  auteur1  ['"Atalla, Nora"']  auteur2  []
isbn different mais titre et auteurs egaux titre1:  la naissance d une heroine  titre2:  la naissance d une etoile  isbn:  ['"9782981478603"']  isbn:  ['"9782981630100"'] 

## Comparaison basée sur le titre

In [32]:
for book1, book2 in couple_books:
    if book1["title"] and book2["title"]:
        dist_bool = Levenshtein.distance(book1['title'], book2['title']) == min(len(book1['title']), len(book2['title'])) / 2
        if dist_bool:
            print("maximum titre equivalent pour /2 titre1: ", book1['title'], ' titre2: ', book2['title'], ' isbn: ',
                book1['isbn_raw'], ' isbn: ', book2['isbn_raw'], " auteur1 ", book1["author_raw"], " auteur2 ", book2["author_raw"])
            break
for book1, book2 in couple_books:
    if book1["title"] and book2["title"]:
        dist_bool = Levenshtein.distance(book1['title'], book2['title']) == min(len(book1['title']), len(book2['title'])) / 3
        if dist_bool:
            print("maximum titre equivalent pour /3 titre1: ", book1['title'], ' titre2: ', book2['title'], ' isbn: ',
                book1['isbn_raw'], ' isbn: ', book2['isbn_raw'], " auteur1 ", book1["author_raw"], " auteur2 ", book2["author_raw"])
            break
for book1, book2 in couple_books:
    if book1["title"] and book2["title"]:
        dist_bool = Levenshtein.distance(book1['title'], book2['title']) == min(len(book1['title']), len(book2['title'])) / 4
        if dist_bool:
            print("maximum titre equivalent pour /4 titre1: ", book1['title'], ' titre2: ', book2['title'], ' isbn: ',
                book1['isbn_raw'], ' isbn: ', book2['isbn_raw'], " auteur1 ", book1["author_raw"], " auteur2 ", book2["author_raw"])
            break
for book1, book2 in couple_books:
    if book1["title"] and book2["title"]:
        dist_bool = Levenshtein.distance(book1['title'], book2['title']) == min(len(book1['title']), len(book2['title'])) / 5
        if dist_bool:
            print("maximum titre equivalent pour /5 titre1: ", book1['title'], ' titre2: ', book2['title'], ' isbn: ',
                book1['isbn_raw'], ' isbn: ', book2['isbn_raw'], " auteur1 ", book1["author_raw"], " auteur2 ", book2["author_raw"])
            break
for book1, book2 in couple_books:
    if book1["title"] and book2["title"]:
        dist_bool = Levenshtein.distance(book1['title'], book2['title']) == min(len(book1['title']), len(book2['title'])) / 6
        if dist_bool:
            print("maximum titre equivalent pour /6 titre1: ", book1['title'], ' titre2: ', book2['title'], ' isbn: ',
                book1['isbn_raw'], ' isbn: ', book2['isbn_raw'], " auteur1 ", book1["author_raw"], " auteur2 ", book2["author_raw"])
            break
for book1, book2 in couple_books:
    if book1["title"] and book2["title"]:
        dist_bool = Levenshtein.distance(book1['title'], book2['title']) == min(len(book1['title']), len(book2['title'])) / 7
        if dist_bool:
            print("maximum titre equivalent pour /7 titre1: ", book1['title'], ' titre2: ', book2['title'], ' isbn: ',
                book1['isbn_raw'], ' isbn: ', book2['isbn_raw'], " auteur1 ", book1["author_raw"], " auteur2 ", book2["author_raw"])
            break       
for book1, book2 in couple_books:
    if book1["title"] and book2["title"]:
        dist_bool = Levenshtein.distance(book1['title'], book2['title']) == min(len(book1['title']), len(book2['title'])) / 8
        if dist_bool:
            print("maximum titre equivalent pour /8 titre1: ", book1['title'], ' titre2: ', book2['title'], ' isbn: ',
                book1['isbn_raw'], ' isbn: ', book2['isbn_raw'], " auteur1 ", book1["author_raw"], " auteur2 ", book2["author_raw"])
            break
for book1, book2 in couple_books:
    if book1["title"] and book2["title"]:
        dist_bool = Levenshtein.distance(book1['title'], book2['title']) == min(len(book1['title']), len(book2['title'])) / 9
        if dist_bool:
            print("maximum titre equivalent pour /9 titre1: ", book1['title'], ' titre2: ', book2['title'], ' isbn: ',
                book1['isbn_raw'], ' isbn: ', book2['isbn_raw'], " auteur1 ", book1["author_raw"], " auteur2 ", book2["author_raw"])
            break
for book1, book2 in couple_books:
    if book1["title"] and book2["title"]:
        dist_bool = Levenshtein.distance(book1['title'], book2['title']) == min(len(book1['title']), len(book2['title'])) / 10
        if dist_bool:
            print("maximum titre equivalent pour /10 titre1: ", book1['title'], ' titre2: ', book2['title'], ' isbn: ',
                book1['isbn_raw'], ' isbn: ', book2['isbn_raw'], " auteur1 ", book1["author_raw"], " auteur2 ", book2["author_raw"])
            break

maximum titre equivalent pour /2 titre1:  les demi-civilises  titre2:  les eminences grises  isbn:  ['""']  isbn:  ['9782894288870']  auteur1  ['"Harvey, Jean-Charles"']  auteur2  [['Yves ThÃ©orÃªt (De (auteur))', ' AndrÃ©-A. Lafrance (De (auteur))']]
maximum titre equivalent pour /3 titre1:  le defi de l amour  titre2:  le masque de l amour  isbn:  ['"2921493330"']  isbn:  ['"2922414140"']  auteur1  ['"Diane" "Gagnon"']  auteur2  ['"François A." "Gouin"']
maximum titre equivalent pour /4 titre1:  soho  titre2:  solo  isbn:  ['"2890055507"']  isbn:  ['"0-88910-449-2"']  auteur1  ['"François" "Piazza"']  auteur2  ['"Tremblay, Michel"']
maximum titre equivalent pour /5 titre1:  garfield - nº 67  titre2:  garfield - na 8  isbn:  ['"9782897510251"']  isbn:  ['"9782895432272"']  auteur1  ['"Jim Davis"']  auteur2  ['"Jim Davis"']
maximum titre equivalent pour /6 titre1:  retour  titre2:  retours  isbn:  ['""']  isbn:  ['""']  auteur1  ['"Rioux, Hélène"']  auteur2  ['"Trottier, Pierre"']
maxi